In [ ]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import psutil
import random
import datetime as datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# rand_sample_csv is a randomized subset (1% the size) of the sample_csv which is ~400k instances
df = pd.read_csv('/Users/samkahr/amli/ad_optimization/notebooks/sam_feature_engineered.csv')


In [51]:
df.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C19,C20,C21,date,time,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
0,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232
1,0,14102100,1005,1,25,7,12,225,12,0,...,167,100075,23,0,0,30,30,30,95,232
2,0,14102100,1005,0,177,225,4,242,15,2,...,427,100000,61,0,0,1499,1499,1499,95,255
3,0,14102100,1005,0,122,125,3,225,12,0,...,39,100084,33,0,0,97,97,97,95,232
4,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232


In [52]:
df.describe()


,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C19,C20,C21,date,time,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
count,4184.000000,4.184000e+03,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,...,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000
mean,0.170172,1.410256e+07,1004.977533,0.283222,160.879780,194.196224,5.920172,195.315249,11.778442,1.478728,...,229.095841,53381.884082,84.231836,4.485421,11.242352,691.010516,691.010516,691.010516,189.910612,201.113050
std,0.375829,2.963423e+02,1.032567,0.489278,87.971659,71.232868,4.339780,56.543830,5.366323,2.883593,...,358.883751,49952.289351,70.673532,2.960619,6.001081,649.791229,649.791229,649.791229,47.062444,59.080199
min,0.000000,1.410210e+07,1002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,33.000000,-1.000000,13.000000,0.000000,0.000000,1.000000,1.000000,1.000000,91.000000,0.000000
25%,0.000000,1.410230e+07,1005.000000,0.000000,108.000000,137.000000,3.000000,199.750000,12.000000,0.000000,...,35.000000,-1.000000,23.000000,2.000000,6.000000,33.000000,33.000000,33.000000,170.000000,203.750000
50%,0.000000,1.410260e+07,1005.000000,0.000000,177.000000,225.000000,4.000000,225.000000,12.000000,0.000000,...,39.000000,100053.500000,61.000000,5.000000,11.000000,720.000000,720.000000,720.000000,204.000000,232.000000
75%,0.000000,1.410281e+07,1005.000000,1.000000,177.000000,225.000000,12.000000,225.000000,12.000000,2.000000,...,169.000000,100084.750000,111.000000,7.000000,16.000000,1499.000000,1499.000000,1499.000000,229.000000,232.000000
max,1.000000,1.410302e+07,1012.000000,7.000000,353.000000,291.000000,13.000000,243.000000,31.000000,13.000000,...,1839.000000,100248.000000,255.000000,9.000000,23.000000,1499.000000,1499.000000,1499.000000,252.000000,256.000000


In [53]:
#dropping unnamed columns
#df = df.drop(columns=['Unnamed: 0'])

df.head()


,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C19,C20,C21,date,time,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
0,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232
1,0,14102100,1005,1,25,7,12,225,12,0,...,167,100075,23,0,0,30,30,30,95,232
2,0,14102100,1005,0,177,225,4,242,15,2,...,427,100000,61,0,0,1499,1499,1499,95,255
3,0,14102100,1005,0,122,125,3,225,12,0,...,39,100084,33,0,0,97,97,97,95,232
4,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232


In [55]:
#finding out how many unique values are in each col

for col in df.columns:
    print(str(col)+ ": " +  "{}".format(len(df[col].unique())) )

click: 2
hour: 240
C1: 5
banner_pos: 5
site_id: 354
site_domain: 292
site_category: 14
app_id: 244
app_domain: 32
app_category: 14
device_id: 660
device_model: 871
device_type: 4
device_conn_type: 4
C14: 675
C15: 4
C16: 5
C17: 279
C18: 4
C19: 59
C20: 105
C21: 54
date: 10
time: 24
device_ip_count: 37
device_id_count: 37
user_count: 37
user_hour_count: 23
user_bagged: 257


In [57]:
#dropping features with high varience

#df = df.drop(columns=['click_bagged'])

In [58]:
# check for missing values
df.isnull().sum()

click               0
hour                0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
date                0
time                0
device_ip_count     0
device_id_count     0
user_count          0
user_hour_count     0
user_bagged         0
dtype: int64

In [149]:
# # separate the date and time
# parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
# df['new_hour'] = df.hour.astype(str).apply(parse_date)
# df['new_hour']

0      2014-10-21 00:00:00
1      2014-10-21 00:00:00
2      2014-10-21 00:00:00
3      2014-10-21 00:00:00
4      2014-10-21 00:00:00
5      2014-10-21 00:00:00
6      2014-10-21 00:00:00
7      2014-10-21 00:00:00
8      2014-10-21 00:00:00
9      2014-10-21 00:00:00
10     2014-10-21 00:00:00
11     2014-10-21 01:00:00
12     2014-10-21 01:00:00
13     2014-10-21 01:00:00
14     2014-10-21 01:00:00
15     2014-10-21 01:00:00
16     2014-10-21 01:00:00
17     2014-10-21 01:00:00
18     2014-10-21 01:00:00
19     2014-10-21 01:00:00
20     2014-10-21 01:00:00
21     2014-10-21 01:00:00
22     2014-10-21 01:00:00
23     2014-10-21 01:00:00
24     2014-10-21 01:00:00
25     2014-10-21 01:00:00
26     2014-10-21 01:00:00
27     2014-10-21 01:00:00
28     2014-10-21 01:00:00
29     2014-10-21 01:00:00
               ...        
4154   2014-10-30 20:00:00
4155   2014-10-30 20:00:00
4156   2014-10-30 21:00:00
4157   2014-10-30 21:00:00
4158   2014-10-30 21:00:00
4159   2014-10-30 21:00:00
4

In [150]:
# df['date'] = [d.date() for d in df['new_hour']]
# df['time'] = [d.time() for d in df['new_hour']]

In [59]:
df.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,C19,C20,C21,date,time,device_ip_count,device_id_count,user_count,user_hour_count,user_bagged
0,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232
1,0,14102100,1005,1,25,7,12,225,12,0,...,167,100075,23,0,0,30,30,30,95,232
2,0,14102100,1005,0,177,225,4,242,15,2,...,427,100000,61,0,0,1499,1499,1499,95,255
3,0,14102100,1005,0,122,125,3,225,12,0,...,39,100084,33,0,0,97,97,97,95,232
4,0,14102100,1005,0,113,227,3,225,12,0,...,39,-1,157,0,0,2,2,2,95,232


In [152]:
# df = df.drop(columns=['new_hour'])

In [153]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,C14,C15,C16,C17,C18,C19,C20,C21,date,time
0,0,0,236,13120299559997056165,0,14102100,1005,0,543a539e,c7ca3108,...,20362,320,50,2333,0,39,-1,157,2014-10-21,00:00:00
1,1,1,259,13447361190641805430,0,14102100,1005,1,17caea14,0dde25ec,...,19950,320,50,1800,3,167,100075,23,2014-10-21,00:00:00
2,2,2,357,14758321504714974000,0,14102100,1005,0,85f751fd,c4e18dd6,...,19743,320,50,2264,3,427,100000,61,2014-10-21,00:00:00
3,3,3,455,16246356889796608406,0,14102100,1005,0,5b08c53b,7687a86e,...,17654,300,250,1994,2,39,100084,33,2014-10-21,00:00:00
4,4,4,540,17569568361412548369,0,14102100,1005,0,543a539e,c7ca3108,...,20362,320,50,2333,0,39,-1,157,2014-10-21,00:00:00


In [60]:
df.dtypes



click               int64
hour                int64
C1                  int64
banner_pos          int64
site_id             int64
site_domain         int64
site_category       int64
app_id              int64
app_domain          int64
app_category        int64
device_id           int64
device_model        int64
device_type         int64
device_conn_type    int64
C14                 int64
C15                 int64
C16                 int64
C17                 int64
C18                 int64
C19                 int64
C20                 int64
C21                 int64
date                int64
time                int64
device_ip_count     int64
device_id_count     int64
user_count          int64
user_hour_count     int64
user_bagged         int64
dtype: object

In [61]:
# #categories = []
df.columns

Index(['click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'date', 'time', 'device_ip_count',
       'device_id_count', 'user_count', 'user_hour_count', 'user_bagged'],
      dtype='object')

In [156]:
# #converting non numerical columns into numerical columns

# categories = ['site_id', 'site_domain','site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
#        'device_ip', 'device_model', 'date', 'time']

# for category in categories:
#   df[category] = df[category].astype('category').cat.codes

In [44]:
for col in df.columns.values:
    print(str(col)+ ": " +  "{}".format(len(df[col].unique())) )


click: 2
hour: 240
C1: 5
banner_pos: 5
site_id: 354
site_domain: 292
site_category: 14
app_id: 244
app_domain: 32
app_category: 14
device_id: 660
device_model: 871
device_type: 4
device_conn_type: 4
C14: 675
C15: 4
C16: 5
C17: 279
C18: 4
C19: 59
C20: 105
C21: 54
date: 10
time: 24
device_ip_count: 37
device_id_count: 37
user_count: 37
user_hour_count: 23
user_bagged: 257


In [62]:
df.head()

features = ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_type', 'device_conn_type', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21', 'date', 'time',
       'device_ip_count', 'device_id_count', 'user_count', 'user_hour_count',
       'user_bagged']

target = ['click']

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.2, random_state = 0)




In [67]:
#running random forest algorithm

from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=1000, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)  





/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [66]:
from sklearn import metrics

print('Log loss:', metrics.log_loss(y_test, y_pred))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 


Log loss: 6.973818392155966


In [68]:
#running scikit learns recursive feature selection
from sklearn.feature_selection import RFE

rfe = RFE(regressor, n_features_to_select=10, step=1, verbose=0)

rfe = rfe.fit(X_train, y_train)

print(rfe.support_)
print(rfe.ranking_)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ True False False  True  True False False False False False False False
 False  True False  True  True  True  True  True False False False  True
 False]
[ 1 14 13  1  1  9  5 11 10 16  8 15 12  1  7  1  1  1  1  1  4  6  3  1
  2]


In [162]:
rfe_features=[ 'site_id', 'site_domain','device_ip', 'device_model', 'C14', 'C19', 'C20', 'C21', 'date', 'time']

bad_features=['[11]C1', '[12]banner_pos', '[5]site_category', '[4]app_id', '[7]app_domain', 
              '[8]app_category', '[3]device_id', '[13]device_type', '[10]device_conn_type', '[14]C15', '[6]C16', '[9]C17', '[2]C18',]

In [163]:
#rerunning Random Forests with rfe_features
X_train, X_test, y_train, y_test = train_test_split(df[rfe_features], df[target], test_size = 0.2, random_state = 0)
  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [164]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.28365949820788533
Mean Squared Error: 0.14509208243727598
Root Mean Squared Error: 0.3809095462669267
